## 模型性能

In [1]:
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import *
import numpy as np
sys.path.append('../data/')
sys.path.append("../")
from load_cifar import load_cifar
from load_olivetti import load_olivetti
from resnet import resnet
from preact_resnet import PreActResNet18

In [2]:
os.listdir('ckpts/')

['olivetti_cap_resnet_4_model.ckpt',
 'cifar_cap_res5_0.2_model.ckpt',
 'cifar_cap_res5_0.4_model.ckpt',
 'cifar_no_preact_resnet_model.ckpt',
 'cifar_cap_res5_0.1_model.ckpt',
 'cifar_cap_preact_resnet_0.1_model.ckpt',
 'olivetti_cap_resnet_2_model.ckpt',
 '.ipynb_checkpoints',
 'olivetti_no_res5_model.ckpt',
 'olivetti_cap_resnet_1_model.ckpt',
 'cifar_cap_preact_resnet_0.4_model.ckpt',
 'olivetti_cap_preact_1_model.ckpt',
 'cifar_no_res5_model.ckpt',
 'olivetti_cap_preact_4_model.ckpt',
 'olivetti_cap_preact_2_model.ckpt',
 'olivetti_no_preact_model.ckpt',
 'cifar_cap_preact_resnet_0.2_model.ckpt']

#### CIFAR10/ResNet34

In [3]:
# 加载数据
def reshape_data(X_train, y_train, X_test):
    # reshape train and subtract mean
    pixel_mean = np.mean(X_train, axis=0)
    X_train -= pixel_mean
    X_test -= pixel_mean
    return X_train, y_train, X_test

def load_data(name='cifar'):
    if name == 'cifar':
        X_train, y_train, X_test, y_test = load_cifar(10)
        X_train = np.dstack((X_train[:, :1024], X_train[:, 1024:2048], X_train[:, 2048:]))
        X_train = X_train.reshape((-1, 32, 32, 3)).transpose(0, 3, 1, 2)
        X_test = np.dstack((X_test[:, :1024], X_test[:, 1024:2048], X_test[:, 2048:]))
        X_test = X_test.reshape((-1, 32, 32, 3)).transpose(0, 3, 1, 2)
    else:
        X_train, y_train, X_test, y_test = load_olivetti()
    
    return X_train, y_train, X_test, y_test

In [4]:
# 加载CIFAR10
X_train, y_train, X_test, y_test = load_data(name='cifar')
X_train, y_train, X_test = reshape_data(X_train, y_train, X_test)
x_data = torch.from_numpy(X_test)
y_data = torch.from_numpy(y_test).long()
cifar10_loader = DataLoader(dataset=TensorDataset(x_data, y_data), 
                          batch_size=512, 
                          shuffle=False, 
                          num_workers=2)

In [5]:
# 加载Olivetti
X_train, y_train, X_test, y_test = load_data(name='olivetti')
X_train, y_train, X_test = reshape_data(X_train, y_train, X_test)
x_data = torch.from_numpy(X_test)
y_data = torch.from_numpy(y_test).long()
olivetti_loader = DataLoader(dataset=TensorDataset(x_data, y_data), 
                          batch_size=512, 
                          shuffle=False, 
                          num_workers=2)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
df_metric = pd.DataFrame(columns=('网络','数据集', '准确率', 'micro-F1', 'macro-F1'))

In [7]:
def metric_model(net, modelpath, data_loader, netname='ResNet34', dataname='CIFAR10'):
    net.load_state_dict(torch.load(modelpath))
    res = []
    y_data = []
    network.eval()
    for i, data in enumerate(data_loader):
        x, y = data
        x_device = x.to(device)
        pred = network(x_device)
        res.append(pred.max(1)[1])
        y_data.append(y.to(device))
    res = torch.cat(res)
    y_data = torch.cat(y_data)
    print(len(res))
    # 准确率
    acc = accuracy_score(res.cpu().numpy(), y_data.cpu().numpy())
    # micro-f1
    micro_f1 = f1_score(res.cpu().numpy(), y_data.cpu().numpy(), average='micro')
    # macro-f1
    macro_f1 = f1_score(res.cpu().numpy(), y_data.cpu().numpy(), average='macro')
    
    global df_metric
    df_metric = df_metric.append({
        '网络': netname, 
        '数据集': dataname,
        '准确率': acc,
        'micro-F1': micro_f1,
        'macro-F1': macro_f1
    }, ignore_index=True)
    df_metric.to_latex(sys.stderr)

In [8]:
# CIFAR10/ResNet34
network = resnet(num_classes=10).to(device)
path = 'ckpts/cifar_no_res5_model.ckpt'
metric_model(network, path, cifar10_loader, netname='ResNet34', dataname='CIFAR10')

10000


\begin{tabular}{lllrrr}
\toprule
{} &        网络 &      数据集 &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &  ResNet34 &  CIFAR10 &  0.9243 &    0.9243 &  0.924126 \\
\bottomrule
\end{tabular}


In [9]:
# CIFAR10/PreActResNet18
network = PreActResNet18(num_classes=10).to(device)
path = 'ckpts/cifar_no_preact_resnet_model.ckpt'
metric_model(network, path, cifar10_loader, netname='PreActResNet18', dataname='CIFAR10')

10000


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &      数据集 &     准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &  CIFAR10 &  0.9243 &    0.9243 &  0.924126 \\
1 &  PreActResNet18 &  CIFAR10 &  0.9441 &    0.9441 &  0.944018 \\
\bottomrule
\end{tabular}


In [10]:
# Olivetti/ResNet34
network = resnet(num_classes=40).to(device)
path = 'ckpts/olivetti_no_res5_model.ckpt'
metric_model(network, path, olivetti_loader, netname='ResNet34', dataname='Olivetti')

120


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &   CIFAR10 &  0.924300 &  0.924300 &  0.924126 \\
1 &  PreActResNet18 &   CIFAR10 &  0.944100 &  0.944100 &  0.944018 \\
2 &        ResNet34 &  Olivetti &  0.908333 &  0.908333 &  0.902738 \\
\bottomrule
\end{tabular}


In [11]:
# Olivetti/PreActResNet18
network = PreActResNet18(num_classes=40).to(device)
path = 'ckpts/olivetti_no_preact_model.ckpt'
metric_model(network, path, olivetti_loader, netname='PreActResNet18', dataname='Olivetti')

120


\begin{tabular}{lllrrr}
\toprule
{} &              网络 &       数据集 &       准确率 &  micro-F1 &  macro-F1 \\
\midrule
0 &        ResNet34 &   CIFAR10 &  0.924300 &  0.924300 &  0.924126 \\
1 &  PreActResNet18 &   CIFAR10 &  0.944100 &  0.944100 &  0.944018 \\
2 &        ResNet34 &  Olivetti &  0.908333 &  0.908333 &  0.902738 \\
3 &  PreActResNet18 &  Olivetti &  0.900000 &  0.900000 &  0.896488 \\
\bottomrule
\end{tabular}
